In [1]:
import sys
import os

# Chemin absolu du répertoire racine du projet
project_root = '/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/'

# Ajouter le répertoire src au chemin de recherche des modules
src = os.path.join(project_root, '.')
sys.path.append(src)



In [2]:
import pandas as pd
import numpy as np
from src.data_concatenation.concatenate import build_macro_table
from src.data_preprocessing.preprocessing import preprocess_dataframe
from src.data_preprocessing.topificator import TopicExtractor
from src.database_management.database_initializer import DatabaseInitializer, wait_for_postgresql
from src.database_management.dataframe_to_transactionalDB import insert_data_from_dataframe
from src.database_management.migrate_to_decisionalDB import migration_to_decisionalDB
from src.database_management.db_models import Base
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from datetime import datetime, timedelta
import psycopg2, os

from tqdm import tqdm

CURRENT_DATE = datetime.now().strftime("%Y-%m-%d")
DATA_PATH = os.path.join(project_root, 'data')
CITIES_PATH = os.path.join(project_root, 'data', 'utils', 'countries_cities-full.json')
RAW_SAVE_PATH = os.path.join(project_root, 'data', 'temp')
PARCKET_PATH = os.path.join(project_root, 'data', 'parquet', CURRENT_DATE)

In [3]:

concatenate_parquets = build_macro_table(DATA_PATH)

# --------------------------------------------------------
# 0.3 Preprocessing of concatenate parquets
# --------------------------------------------------------
# df = preprocess_dataframe(concatenate_parquets)

len(concatenate_parquets.columns)

17

In [ ]:

# --------------------------------------------------------
# 0.3 Preprocessing of concatenate parquets
# --------------------------------------------------------
df = preprocess_dataframe(concatenate_parquets)

In [17]:
# all df
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# df


In [6]:
df.isna().sum()

Country                      0
Town                         0
Bank_Name                    0
Bank_Phone_number            0
Bank_Address                 0
Bank_Website                 0
Reviewer_Name                0
Reviewer_Star                0
Reviewer_Text                0
Reviewer_Publish_Date        0
Reviewer_Like_Reaction       0
Reviewer_Profile_Link        0
Reviewer_Owner_Reply         1
Reviewer_Owner_Reply_Date    0
Topic                        0
Sentiment                    0
Sub_Topic                    0
dtype: int64

In [7]:
df.head()

,Country,Town,Bank_Name,Bank_Phone_number,Bank_Address,Bank_Website,Reviewer_Name,Reviewer_Star,Reviewer_Text,Reviewer_Publish_Date,Reviewer_Like_Reaction,Reviewer_Profile_Link,Reviewer_Owner_Reply,Reviewer_Owner_Reply_Date,Topic,Sentiment,Sub_Topic
0,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Mohamed Nador,4,Berger,2016-08-09 02:47:38.649920,0,https://www.google.com/maps/contrib/1165746157...,Nothing,1970-01-01,service client,Negative,"[malhonnête, incompétent]"
1,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Mohamed Nador,4,Berger,2016-08-09 02:47:38.649920,0,https://www.google.com/maps/contrib/1165746157...,Nothing,1970-01-01,temps d'attente,Negative,[trop long]
2,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Mohamed Nador,4,Berger,2016-08-09 02:47:38.649920,0,https://www.google.com/maps/contrib/1165746157...,Nothing,1970-01-01,frais bancaires,Negative,[élevés]
3,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Mohamed Nador,4,Berger,2016-08-09 02:47:38.649920,0,https://www.google.com/maps/contrib/1165746157...,Nothing,1970-01-01,application mobile,Positive,[intuitive]
4,Morocco,Nador,Banque Populaire,Not available,"53Hg+Fvc, Bd Mohamed Ameziane Errifi, Nador",not available,Hafaimya,3,Expérience neutre,2021-08-08 02:47:38.649927,0,https://www.google.com/maps/contrib/1027703754...,Nothing,1970-01-01,service client,Neutre,"[normaux, aucun problème spécifique]"


In [ ]:

# --------------------------------------------------------
# 0.5 Initialization of database
# --------------------------------------------------------
# Utilisation de la DatabaseInitializer
ADMIN_USER = 'postgres'
ADMIN_PASSWORD = 'postgres'
HOST = 'postgresql'
PORT = '5432'
DB_USER = 'conite'
DB_PASSWORD = 'conite_password'
DB_NAME = 'bank_reviews'

# --------------------------------------------------------------------------
# Database Initialization
# --------------------------------------------------------------------------
db_initializer = DatabaseInitializer(ADMIN_USER, ADMIN_PASSWORD, HOST, PORT)

db_initializer.create_database_and_user(DB_USER, DB_PASSWORD, DB_NAME)

# ------------------------------------------------------------------------------
# Configuration de la base de donnees
# ------------------------------------------------------------------------------
DB_URI = f'postgresql://{DB_USER}:{DB_PASSWORD}@{HOST}:5432/{DB_NAME}'

# ------------------------------------------------------------------------------
# Initialize database and create tables
# ------------------------------------------------------------------------------

engine = create_engine(DB_URI)
Session = sessionmaker(bind=engine)
session = Session()
wait_for_postgresql(host=HOST, port=PORT, user=DB_USER, password=DB_PASSWORD, dbname=DB_NAME)
Base.metadata.create_all(engine)

conn_params = {
    'dbname': DB_NAME,
    'user': DB_USER,
    'password': DB_PASSWORD,
    'host': HOST,
    'port': PORT
}
with psycopg2.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        """
        cur.execute(query)
        existing_tables = cur.fetchall()
        existing_tables = [table[0] for table in existing_tables]
        print("\033[1;92mTABLES CREATED SUCCESSFULLY.\033[0m \033[1;94m" + str(existing_tables) + "\033[0m")

# --------------------------------------------------------
# 0.6 Insertion of data in database
# --------------------------------------------------------
insert_data_from_dataframe(df, session)
migration_to_decisionalDB(DB_NAME, DB_PASSWORD, session)
# session.commit()

In [3]:
!python main.py

PULLING:  Morocco
  0%|                                                     | 0/1 [00:04<?, ?it/s]
Traceback (most recent call last):
  File "/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/src/main.py", line 231, in <module>
    main()
  File "/home/conite/Documents/WORKSPACE/PROJECTS/BI-PROJECT/BANK-REVIEWS-INTELLIGENCE/src/main.py", line 61, in main
    time.sleep(20)
KeyboardInterrupt
